In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import dep.arcturis as arc
import gpt as kadabra

from openai import OpenAI

client = OpenAI()

language = 'Python'

options = webdriver.FirefoxOptions()
driver = webdriver.Firefox(options=options)
driver.get("https://leetcode.com/accounts/login/")

In [ ]:
#initializaion, login and problem select

def wait_and_find_element(driver, by, value):
        return WebDriverWait(driver, 10).until(EC.presence_of_element_located((by, value)))

def login_and_select(driver):
    login_field = wait_and_find_element(driver, By.ID, "id_login")
    login_field.send_keys(arc.e)    

    password_field = wait_and_find_element(driver, By.ID, "id_password")
    password_field.send_keys(arc.p)

    EC.element_to_be_clickable((By.XPATH, "//div[@class='btn-content-container__2HVS']/span[text()='Sign In']"))
    WebDriverWait(driver, 10).until(EC.invisibility_of_element_located((By.ID, "initial-loading")))

    # Find the "Sign In" button
    sign_in_button = wait_and_find_element(driver,By.XPATH, "//div[@class='btn-content-container__2HVS']/span[text()='Sign In']")

    driver.execute_script("arguments[0].scrollIntoView(true);", sign_in_button)

    time.sleep(5)
    sign_in_button.click()
    time.sleep(3)


    problems = wait_and_find_element(driver, By.XPATH,"//a[@href='/problemset/all/']")
    problems.click()

    pickone = wait_and_find_element(driver, By.XPATH,"//div[contains(@class, 'ml-auto')]")
    # Click the button  
    pickone.click()

In [ ]:
#pophandler loop
def click_next_button(driver):
    #find all of the elements in the navbar with "cursor-pointer" and click the 3rd one (next problem)
    navbar = wait_and_find_element(driver, By.CLASS_NAME,"z-nav-1")
    buttons = navbar.find_elements(By.CLASS_NAME,"cursor-pointer")
    buttons[2].click()  

def handle_subscribe_popup(driver):
    while True:
        try:
            # Check for the subscribe popup
            subscribe_popup = WebDriverWait(driver, 4).until(                
                EC.visibility_of_element_located((By.XPATH, "//div[contains(text(), 'Subscribe to unlock.')]"))
            )

            click_next_button(driver)
                

        except:
            # If the subscribe popup is not found, break out of the loop
            break

solution = ''
def solution_input(driver, language):
    #extracts the description and pre-formatted code, inserts into a gpt prompt
    #clears the text window and inserts the solution

    global solution
    
    description_element = wait_and_find_element(driver, By.CLASS_NAME, 'elfjS')
    description_text = description_element.text

    code_block_element = wait_and_find_element(driver, By.CSS_SELECTOR, '.view-lines')
    code_block_text = code_block_element.text    

    solution = kadabra.main(f"I would like you to solve a problem in {language}, please only return code, do not include any additional text and do not add comments within the code, do not use any shortcuts for brevity (write the entire solution) and do not add ```cpp or ``` at the beginning or end of the solution. Here is the description of the problem: {description_text} and use this format to provide the solution {code_block_text}")

    charstoclear = len(code_block_element.text)

    textwindow = wait_and_find_element(driver, By.CLASS_NAME, "view-lines.monaco-mouse-cursor-text")
    textwindow.click()

    #press down arrow the length of code block text in case text.window.click() is not at the bottom (to start clearing)
    for _ in range(len(code_block_text.split('\n'))+7):
        textwindow = wait_and_find_element(driver, By.CLASS_NAME, "view-lines.monaco-mouse-cursor-text")
        ActionChains(driver).move_to_element(textwindow).send_keys(Keys.ARROW_DOWN).perform()

    #press backspace double the amount of characters in the string of code_block_text to clear the window
    for _ in range(charstoclear + charstoclear):
        textwindow = wait_and_find_element(driver, By.CLASS_NAME, "view-lines.monaco-mouse-cursor-text")
        ActionChains(driver).move_to_element(textwindow).send_keys(Keys.BACKSPACE).perform()

    litch = solution.split('\n')

    for line in litch:
        ActionChains(driver).move_to_element(textwindow).send_keys(line).perform()
        ActionChains(driver).move_to_element(textwindow).send_keys(Keys.ENTER).perform()
    
    
def click_submit(driver):
    #click submit 
    submit = wait_and_find_element(driver,By.CSS_SELECTOR, "button[data-e2e-locator='console-submit-button']")
    submit.click()

def click_format_button(driver):
    time.sleep(1)
    thiscontainer = WebDriverWait(driver, 1).until(
        EC.visibility_of_element_located((By.CLASS_NAME, "flex.h-8.items-center.justify-between.border-b.p-1.border-border-quaternary"))
    )
    # click format code button
    button = thiscontainer.find_element(By.CSS_SELECTOR, "button[data-state='closed']")
    button.click()

def scrolldown(driver):   
    textwindow = wait_and_find_element(driver, By.CLASS_NAME, "view-lines.monaco-mouse-cursor-text")
    textwindow.click()
    # "scroll" down to the bottom
    for _ in range(len(solution.split('\n')) + 25):
        textwindow = wait_and_find_element(driver, By.CLASS_NAME, "view-lines.monaco-mouse-cursor-text")
        ActionChains(driver).move_to_element(textwindow).send_keys(Keys.ARROW_DOWN).perform()

def remove_brackets(driver):
    textwindow = wait_and_find_element(driver, By.CLASS_NAME, "view-lines.monaco-mouse-cursor-text")    
    extras = solution.count('}')
    for _ in range((extras * 2) - 1):
        ActionChains(driver).move_to_element(textwindow).send_keys(Keys.BACKSPACE).perform()


In [ ]:

login_and_select(driver)

while True:

    click_next_button(driver)
    handle_subscribe_popup(driver)
    solution_input(driver,language)
    click_format_button(driver)
    scrolldown(driver)
    remove_brackets(driver)
    click_submit(driver)    
